In [12]:
import pandas as pd
import dlib
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
import math

# Load the model

In [13]:
loaded_model = pickle.load(open('faceshape_trial_v4.pkl', 'rb'))
# result = loaded_model.score(x, y)
# print(result)
# print(loaded_model.predict(np.array(df3.iloc[1]).reshape(1, -1)))

# Testing on the human face

In [14]:
# real_image_dir = '/Users/srishtigoel/Downloads/imagesForDemo/ri_*.png'
real_image_dir = '/Users/srishtigoel/Downloads/raw_image_female/raw_image/*'

In [15]:
import glob
import cv2
import dlib
import scipy
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

predictor_path = "/Users/srishtigoel/Downloads/dlibmodel/shape_predictor_68_face_landmarks.dat"
from scipy.spatial import ConvexHull, convex_hull_plot_2d
SPLINEPOINTS=50

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

In [16]:
LEFT_EYE_INDICES = [36, 37, 38, 39, 40, 41]
RIGHT_EYE_INDICES = [42, 43, 44, 45, 46, 47]

def rect_to_tuple(rect):
    left = rect.left()
    right = rect.right()
    top = rect.top()
    bottom = rect.bottom()
    return left, top, right, bottom

def extract_eye(shape, eye_indices):
    points = map(lambda i: shape.part(i), eye_indices)
    return list(points)

def extract_eye_center(shape, eye_indices):
    points = extract_eye(shape, eye_indices)
    xs = map(lambda p: p.x, points)
    ys = map(lambda p: p.y, points)
    return sum(xs) // 6, sum(ys) // 6

def extract_left_eye_center(shape):
    return extract_eye_center(shape, LEFT_EYE_INDICES)

def extract_right_eye_center(shape):
    return extract_eye_center(shape, RIGHT_EYE_INDICES)

def get_biggest_bounding_box(bounding_boxes):
    index_of_bounding_box = -1
    max_area = -1
    for i, bounding_box_each in enumerate(bounding_boxes):
        height = bounding_box_each.height()
        width = bounding_box_each.width()
        area = height*width
        if area > max_area:
            area = max_area
            index_of_bounding_box = i
    if index_of_bounding_box == -1:
        return None
    else:
        return index_of_bounding_box
def angle_between_2_points(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    tan = (y2 - y1) / (x2 - x1)
    return np.degrees(np.arctan(tan))

def get_rotation_matrix(p1, p2):
    angle = angle_between_2_points(p1, p2)
    x1, y1 = p1
    x2, y2 = p2
    xc = (x1 + x2) // 2
    yc = (y1 + y2) // 2
    M = cv2.getRotationMatrix2D((xc, yc), angle, 1)
    return M
def correct_pose(image):
    bounding_boxes_all = detector(image, 0)
    bounding_box_index = get_biggest_bounding_box(bounding_boxes_all)
    if bounding_box_index is None:
        return None
    bounding_box = bounding_boxes_all[bounding_box_index]
    face_points = predictor(image, bounding_box)
    left_eye = extract_left_eye_center(face_points)
    right_eye = extract_right_eye_center(face_points)
    M = get_rotation_matrix(left_eye, right_eye)
    w,h,c = image.shape
    image_max_size_possible = int(math.sqrt(w*w + h*h))
    offset_x = int((image_max_size_possible - image.shape[0])/2)
    offset_y = int((image_max_size_possible - image.shape[1])/2)
    dst_image = np.ones((image_max_size_possible, image_max_size_possible, 3),  dtype='uint8') * 255
    dst_image[offset_x:(offset_x + image.shape[0]), offset_y:(offset_y + image.shape[1]), :] = image
    pose_corrected_image = cv2.warpAffine(src=dst_image, dsize=(image_max_size_possible, image_max_size_possible), M=M, borderValue=[255,255,255])
    return pose_corrected_image

def findface(image):
    bounding_boxes_all = detector(image, 1)
    bounding_box_index = get_biggest_bounding_box(bounding_boxes_all)
    if bounding_box_index is None:
        return None
    bounding_box = bounding_boxes_all[bounding_box_index]
    face_points = predictor(image, bounding_box)
    return face_points

def find_hull_im(im):
    shape = findface(im)
    x=[]
    y=[]
    for i in range(2,15):
        x.append(shape.part(i).x)
        y.append(shape.part(i).y)

    x = np.asarray(x)
    y = np.asarray(y)
    dist = np.sqrt((x[:-1] - x[1:])**2 + (y[:-1] - y[1:])**2)
    dist_along = np.concatenate(([0], dist.cumsum()))

    spline, u = scipy.interpolate.splprep([x, y], u=dist_along, s=0)
    
    # resample it at smaller distance intervals
    interp_d = np.linspace(dist_along[0], dist_along[-1], SPLINEPOINTS)
    interp_x, interp_y = scipy.interpolate.splev(interp_d, spline)
    # plot(interp_x, interp_y, '-o')
    c =[]
    for i in range(len(interp_x)):
        c.append([interp_x[i],interp_y[i]])
    c = np.asarray(c)

    hull1 = ConvexHull(c)
    return hull1,c,shape

def normalize_points(np_array):
    max_point = np.max(np_array, axis=0)
    min_point = np.min(np_array, axis=0)
    height = max_point - min_point
    return (np_array - min_point)/height

In [17]:

def hconcat_resize_min(im_list, interpolation=cv2.INTER_CUBIC):
    h_min = min(im.shape[0] for im in im_list)
    im_list_resize = [cv2.resize(im, (int(im.shape[1] * h_min / im.shape[0]), h_min), interpolation=interpolation)
                      for im in im_list]
    return cv2.hconcat(im_list_resize)


import operator
def find_polyfit_coeff(X,Y,degree):
    
    polynomial_features= PolynomialFeatures(degree=degree)
    x_poly = polynomial_features.fit_transform(X)

    model = LinearRegression(normalize=True)
    model.fit(x_poly,Y)
    # model.score(X, Y)
    y_poly_pred = model.predict(x_poly)


#     plt.scatter(X, Y, s=10)
#     # sort the values of x before line plot
#     sort_axis = operator.itemgetter(0)
#     sorted_zip = sorted(zip(X,y_poly_pred), key=sort_axis)
#     x, y_poly_pred = zip(*sorted_zip)
#     plt.plot(x, y_poly_pred, color='m')
#     plt.show()
    return model.coef_

#     import operator
#     rmse = np.sqrt(mean_squared_error(Y,y_poly_pred))
#     r2 = r2_score(Y,y_poly_pred)
#     print(rmse)
#     print(r2)

def find_ratio(shape):
    x=[]
    y=[]
    for i in range(17):
        x.append(shape.part(i).x)
        y.append(shape.part(i).y)
    x = np.asarray(x)
    y = np.asarray(y)


    left = int(min(x))
    right = int(max(x))
    top = int(min(y))
    bottom = int(max(y))
    w = right - left
    h = bottom-top

    face_ratio_x = w/(h*1.16)
    face_ratio_x = min(face_ratio_x,1.05)
    face_ratio_x = max(face_ratio_x,0.96)
    face_ratio_x = round(face_ratio_x, 2)
    
    return face_ratio_x

In [18]:

image_nm=[]
fp=[]

image_name_list=[]
Side_Face=[]
cheekup=[]
jawout=[]
cheekout=[]
jawup=[]


for image_path in glob.glob(real_image_dir)[:1]:
    print(image_path)
    im = cv2.imread(image_path)
    
    image = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    image = correct_pose(image)

    if image is None:
        continue

    hull_image,c,facepoints = find_hull_im(image)
    image_face_width_height_ratio = find_ratio(facepoints)

    if(image_face_width_height_ratio < 1):
        image_face_width_height_ratio = (1-image_face_width_height_ratio)*750
    elif(image_face_width_height_ratio > 1):
        image_face_width_height_ratio = (image_face_width_height_ratio-1)*-400
    else:
        image_face_width_height_ratio=0
    print(image_face_width_height_ratio)

#     hul_fs,c = find_hull(image_path, top, right, threshhold)
    xh=[]
    yh=[]

    print(len(hull_image.vertices))
    for i in range(len(hull_image.vertices)):
        xh.append(int(c[hull_image.vertices[i],0]))
        yh.append(int(c[hull_image.vertices[i],1]))
#         plt.plot(c[hul_fs.vertices[i],0], c[hul_fs.vertices[i],1], 'ro')
        cv2.circle(image,(xh[i],yh[i]),2,(255,0,0),5)
    
#     print(c[i][0])
#     for i in range(len(c)):
#         xh.append(c[i][0])
#         yh.append(c[i][1])
#         cv2.circle(im,(int(c[i][0]),int(c[i][1])),2,(255,0,0),5)
#     plt.imshow(im)

    # xh,yh
    xh = np.asarray(xh)
    yh = np.asarray(yh)
    xh = normalize_points(xh)
    yh = normalize_points(yh)
    yh = yh*-1
    X = xh[:, np.newaxis]
    Y = yh[:, np.newaxis]

    coeff = find_polyfit_coeff(X,Y,16)

    
    print('coeff is :',coeff)
    preds = loaded_model.predict(np.array(coeff).reshape(1, -1))[0]
    print(preds)
    
    #	Side_Face(max)	L_Cheek_Up	L_Cheek_Out	L_Jaw_Up	L_Jaw_Out
    image_name_list.append(image_path.split('/')[-1])
    Side_Face.append(image_face_width_height_ratio)#preds[0])
    cheekup.append(preds[0])
    jawout.append(preds[3])
    cheekout.append(preds[1])
    jawup.append(preds[2])


/Users/srishtigoel/Downloads/raw_image_female/raw_image/3adaa22e-0d6a-46e4-b645-f3d5a3896e33U98w_daQwUFY-zQU.png
-20.000000000000018
46
coeff is : [[ 0.00000000e+00 -1.65353641e+01  3.21377313e+02 -5.31750566e+03
   6.04869661e+04 -4.64352061e+05  2.46573482e+06 -9.31202745e+06
   2.55406929e+07 -5.14868236e+07  7.64803327e+07 -8.30765987e+07
   6.46644870e+07 -3.46928203e+07  1.19456184e+07 -2.28970504e+06
   1.69987027e+05]]
[-17.1        -23.66666667 -23.         -20.26666667  -8.06666667]


In [19]:
df = pd.DataFrame({'image_name':image_name_list,'Side_Face(max)':Side_Face,'L_Cheek_Up':cheekup,'R_Cheek_Up':cheekup,'L_Cheek_Out':cheekout,'R_Cheek_Out':cheekout,
                'L_Jaw_Up':jawup,'R_Jaw_Up':jawup,'L_Jaw_Out':jawout,'R_Jaw_Out':jawout })

df

,image_name,Side_Face(max),L_Cheek_Up,R_Cheek_Up,L_Cheek_Out,R_Cheek_Out,L_Jaw_Up,R_Jaw_Up,L_Jaw_Out,R_Jaw_Out
0,3adaa22e-0d6a-46e4-b645-f3d5a3896e33U98w_daQwU...,-20.0,-17.1,-17.1,-23.666667,-23.666667,-23.0,-23.0,-20.266667,-20.266667


In [20]:
df.to_csv('supervised_results_v5_testset_female.csv')

In [909]:
!mkdir results_supervised_v5_testset_female

In [21]:
screenshotpath = './screenshots_supervised_testset_v5_female/'
resultpath = './results_supervised_v5_testset_female/'
for image_path in glob.glob(real_image_dir):
 try:
    print(image_path)
    image = cv2.imread(image_path)
#     image_fp2 = cv2.imread('./results_v2/' + image_path.split('/')[-1])
    image_fp = cv2.imread(screenshotpath + image_path.split('/')[-1])
    image_stack = hconcat_resize_min([image,image_fp])
    
    cv2.imwrite(resultpath + image_path.split('/')[-1],image_stack)
 except:
    print('fail')

/Users/srishtigoel/Downloads/raw_image_female/raw_image/3adaa22e-0d6a-46e4-b645-f3d5a3896e33U98w_daQwUFY-zQU.png
/Users/srishtigoel/Downloads/raw_image_female/raw_image/3b207661-7013-42a8-a071-d5462c3c9188U58Y9bkTDBgADXYh.png
/Users/srishtigoel/Downloads/raw_image_female/raw_image/3ba48e23-0848-4d8f-892a-c74653d71713VhEv_07sU29u8G9a.png
/Users/srishtigoel/Downloads/raw_image_female/raw_image/3bac5652-8139-441e-b0a4-d4586a6e1789WpIfwHFwZRAJRKv_.png
/Users/srishtigoel/Downloads/raw_image_female/raw_image/3b848c3a-b03f-4766-9ba2-17c9b116c2c8VK9wXbkTDHach41f.png
/Users/srishtigoel/Downloads/raw_image_female/raw_image/3ac91554-8000-4c50-8146-6ba50a8bfd5bU8-WVbwsRFdOA378.png
/Users/srishtigoel/Downloads/raw_image_female/raw_image/3bf16027-5a1e-41c7-aef6-07d11b60b4a7WgRhKEDBKxB2pEwq.png
/Users/srishtigoel/Downloads/raw_image_female/raw_image/3bc738f2-47ca-4edf-84df-e8eaa9e28b94VCgF2WSBJ0p5aJw3.png
/Users/srishtigoel/Downloads/raw_image_female/raw_image/3b4f1d82-642f-4ed6-8eda-1a3715bf27d8U70K

/Users/srishtigoel/Downloads/raw_image_female/raw_image/3b4fa150-245b-466b-b5e3-ba228ee2021cWBTXvZ3KYndwiy7F.png
/Users/srishtigoel/Downloads/raw_image_female/raw_image/3b107ccb-8cb7-4057-a14e-28423adca35fV5V8-GSBJ45Pcm6s.png
/Users/srishtigoel/Downloads/raw_image_female/raw_image/3be2ba0d-49e3-477f-b095-1f47ca0adb8dWs3St8dBoQ_6bBVT.png
/Users/srishtigoel/Downloads/raw_image_female/raw_image/3b09297d-571f-4570-bb53-ead3fe8cb41bVBCHFtaQwVcLjAey.png
/Users/srishtigoel/Downloads/raw_image_female/raw_image/3c196c12-e922-4104-8b6a-794143e5f97eUoSJ_rkTDEYaNV7D.png
/Users/srishtigoel/Downloads/raw_image_female/raw_image/3b17e039-c47c-4ee3-a6e7-da605f2f401fXAofpnQE0UkqDCoK.png
/Users/srishtigoel/Downloads/raw_image_female/raw_image/3c09a4d3-bd4e-44b2-bd2b-910d2d79b4caXCUMYM84dBMRpdKc.png
/Users/srishtigoel/Downloads/raw_image_female/raw_image/3bab1875-75cb-4c6a-9c39-94a355f8f13bWqTu1DxxIV8mY8YK.png
/Users/srishtigoel/Downloads/raw_image_female/raw_image/3c26a942-08f9-437f-8545-0e2155128b29We8D